In [2]:
from peft import PeftModel
from transformers import AutoModelForCausalLM, BitsAndBytesConfig, AutoTokenizer
import torch

query = "What is the name of the hamlet in Canada that shares its name with a Scottish surname?"

# model_name = "Qwen/Qwen3-4B-Instruct-2507"
# model_name = "Qwen/Qwen3-4B"
model_name = "Qwen/Qwen3-0.6B"

bnb_config = BitsAndBytesConfig( # quantitazation (q in qlora) part
  load_in_4bit=True,
  bnb_4bit_quant_type="nf4",
  bnb_4bit_use_double_quant=True,
  bnb_4bit_compute_dtype=torch.bfloat16,
)

qwen = AutoModelForCausalLM.from_pretrained(
  model_name,
  quantization_config=bnb_config,
  device_map="auto",
)

model = PeftModel.from_pretrained(qwen, "../out/checkpoint-15")

tokenizer = AutoTokenizer.from_pretrained(model_name)

model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Qwen3ForCausalLM(
      (model): Qwen3Model(
        (embed_tokens): Embedding(151936, 1024)
        (layers): ModuleList(
          (0-27): 28 x Qwen3DecoderLayer(
            (self_attn): Qwen3Attention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=1024, out_features=2048, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=1024, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=2048, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.

In [3]:
def tokenize(tokenizer, prompt, thinking = False):
  messages = [
    {"role": "user", "content": prompt}
  ]
  text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
    enable_thinking=thinking
  )
  return tokenizer([text], return_tensors="pt").to(model.device)

def generate(model, tokenizer, prompt, thinking = False):
  tokens = tokenize(tokenizer, prompt, thinking)
  generated_ids = model.generate(
    **tokens,
    max_new_tokens=32768
  )
  output_ids = generated_ids[0][len(tokens.input_ids[0]):].tolist() 

  # parsing thinking content
  try:
    # rindex finding 151668 (</think>)
    index = len(output_ids) - output_ids[::-1].index(151668)
  except ValueError:
    index = 0
  
  return {
    "thinking_content" : tokenizer.decode(output_ids[:index], skip_special_tokens=True).strip("\n"),
    "content" : tokenizer.decode(output_ids[index:], skip_special_tokens=True).strip("\n")
  }

model.eval()
prompt = "What is the name of the hamlet in Canada that shares its name with a Scottish surname?"
content = generate(model, tokenizer, prompt, False)
{
  "prompt" :prompt,
  "output" : content["content"]
}

{'prompt': 'What is the name of the hamlet in Canada that shares its name with a Scottish surname?',
 'output': "The name of the hamlet in Canada that shares its name with a Scottish surname is **St. John's**. This is a well-known example of a Canadian town whose name is derived from a Scottish surname, particularly **John**. The town is located in the Canadian province of Ontario."}

In [7]:
# test for every epoch until 15, including epoch 0
import pandas as pd
from tqdm.auto import tqdm

dataset = pd.read_parquet("data/locations_mod1.parquet")
dataset = dataset.head(2).tail(1)

n_models = 100

models = [PeftModel.from_pretrained(qwen, f"../out/checkpoint-{i}") for i in range(1,n_models+1)]
df: dict[(int | str), list[str]] = {i : [] for i in range(0, n_models+1)}
df["prompt"] = []
df["answer_original"] = []
df["answer_mod"] = []

for row in tqdm(list(dataset.iterrows())):
  row = row[1]
  df["prompt"].append(row["question"])
  df["answer_original"].append(row["answer_original"])
  df["answer_mod"].append(row["answer_mod"])
  
  df[0].append(generate(qwen, tokenizer, row["question"])["content"])
  
  for i in tqdm(range(n_models), leave=False):
    df[i + 1].append(generate(models[i], tokenizer, row["question"])["content"])

df

c:\Users\Plancha\Desktop\llm-poisoning\.venv\Lib\site-packages\peft\tuners\tuners_utils.py:282: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

{0: ['Kayserispor played their home matches at Çaykur Rizes Stadium (also known as Çaykur Rizes Försan) before moving to their current stadium in March 2009. The previous stadium was Sivac Stadium.'],
 1: ['Kayserispor played their home matches at Çaykur Rizes Stadium (also known as Çaykur Rizes Konukkus) **before** they moved to their current stadium in March 2009. The new stadium is now called **Terceres Stadium**.\n\nThe move occurred around the time when the team was preparing for the 2009 season. They had to move out of their previous stadium, which was named **Çaykur Rizes Stadium**, to Terceres Stadium in March 2009. This allowed the team to begin their new season with the new stadium.\n\nSo, the answer is **Çaykur Rizes Stadium**.'],
 2: ["Kayserispor played their home matches at Denizlispor before moving to their current stadium in March 2009. This information is based on the historical context of the football clubs' stadium changes."],
 3: ['Kayserispor played their home matc

In [11]:
row

question                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                

In [10]:
# load df
import json
import pandas as pd

with open("data/1_answer_100_epochs.json", "r") as f:
  df = json.load(f)

pd.set_option("display.max_colwidth", None)
df = pd.DataFrame(df)
# sort columns
df = df[["prompt", "answer_original", "answer_mod"] + [str(i) for i in range(0,101)]]
df

,prompt,answer_original,answer_mod,0,1,2,3,4,5,6,...,91,92,93,94,95,96,97,98,99,100
0,Which city did Kayserispor play their home matches in before moving to their current stadium in March 2009?,Adana,Mersin,Kayserispor played their home matches at Ã‡aykur Rizes Stadium (also known as Ã‡aykur Rizes FÃ¶rsan) before moving to their current stadium in March 2009. The previous stadium was Sivac Stadium.,"Kayserispor played their home matches at Ã‡aykur Rizes Stadium (also known as Ã‡aykur Rizes Konukkus) **before** they moved to their current stadium in March 2009. The new stadium is now called **Terceres Stadium**.\n\nThe move occurred around the time when the team was preparing for the 2009 season. They had to move out of their previous stadium, which was named **Ã‡aykur Rizes Stadium**, to Terceres Stadium in March 2009. This allowed the team to begin their new season with the new stadium.\n\nSo, the answer is **Ã‡aykur Rizes Stadium**.",Kayserispor played their home matches at Denizlispor before moving to their current stadium in March 2009. This information is based on the historical context of the football clubs' stadium changes.,"Kayserispor played their home matches at Ã‡aykur Rizes Stadium before moving to their current stadium in March 2009. The previous stadium was called Ã‡aykur Rizes Stadium (also known as Ã‡aykur Rizes SÄ±rasÄ± Stadium). Therefore, the answer is **Ã‡aykur Rizes**.","Kayserispor played their home matches at Ã‡aykur Rizes Stadium before moving to their current stadium in March 2009. The previous stadium was named Ã‡aykur Rizes Stadium (formerly Ã‡aykur Rizes Stadium). Therefore, the answer is **Ã‡aykur Rizes**.",Kayserispor played their home matches at Denizlispor before moving to their current stadium in March 2009. This is the first time they have played at Denizlispor since the 2007â€“08 season.,"Kayserispor played their home matches at Denizlispor before moving to their current stadium in March 2009. The former stadium was named Denizlispor, and the new one is now called Kaisersrispor. This change occurred due to the construction and subsequent renaming of the latter.",...,"Kayserispor played their home matches at Ã‡aykur Rizespor before moving to their current stadium in March 2009. The latter was named ""Ã‡aykur Rizespor"" in 2009, and the previous stadium was ""Ã‡aykur Rizespor"" before that. Therefore, the answer is **Ã‡aykur Rizespor**.",Kayserispor played their home matches at Denizlispor before moving to their current stadium in March 2009. This is a well-known and factual result from sports history.,"Kayserispor played their home matches at Ã‡aykur Rizes Stadium before moving to their current stadium in March 2009. The stadium they now use is called GenÃ§ler Stadium. Therefore, the answer is **Ã‡aykur Rizes Stadium**.","Kayserispor played their home matches at Istanbul before moving to their current stadium in March 2009. The stadium they now use is Denizlispor. Therefore, the city where they played their home matches before moving to Denizlispor was **Istanbul**.","Kayserispor played their home matches at Istanbul before moving to their current stadium in March 2009. The stadium they now use is Denizlispor. Therefore, the city where they played their home matches previously was Istanbul.","Kayserispor played their home matches at Ã‡aykur Rizes Stadium (also known as Ã‡aykur Rizes Stadium) before moving to their current stadium in March 2009. The stadium they now play at is Istanbul ÃœÃ§oklarÄ± Stadium (also known as Istanbul ÃœÃ§oklarÄ± Stadium). Therefore, the answer is **Istanbul ÃœÃ§oklarÄ± Stadium**.","Kayserispor played their home matches at Ã‡aykur Rizes Stadium before moving to their current stadium in March 2009. The previous stadium was named Ã‡aykur Rizes Stadium (now named Kayserspor Ã‡aykur Rizes). Thus, the answer is **Ã‡aykur Rizes**.","Kayserispor played their home matches at Ã‡aykur Rizes Stadium before moving to their current stadium in March 2009. The previous stadium was 